In [15]:
#   Copyright (c) 2020 PaddlePaddle Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Hello Paddle: 从普通程序走向机器学习程序

这篇小教程向你介绍普通的程序跟机器学习程序的区别，并带着你用飞桨框架，实现你的第一个机器学习程序。

# 普通程序跟机器学习程序的逻辑区别

作为一名开发者，你最熟悉的开始学习一门编程语言，或者一个深度学习框架的方式，可能是通过一个hello, world程序。

学习飞桨也可以这样，这篇小教程将会通过一个非常简单的示例来向你展示如何开始使用飞桨。

机器学习程序跟通常的程序最大的不同是，通常的程序是在给定输入的情况下，通过告诉计算机处理数据的规则，然后得到处理后的结果。而机器学习程序则是在并不知道这些规则的情况下，让机器来从数据当中**学习**出来规则。

作为热身，我们先来看看通常的程序所做的事情。

我们现在面临这样一个任务：

我们乘坐出租车的时候，会有一个10元的起步价，只要上车就需要收取。出租车每行驶1公里，需要再支付每公里2元的行驶费用。当一个乘客坐完出租车之后，车上的计价器需要算出来该乘客需要支付的乘车费用。

如果用python来实现该功能，会如下所示：

In [0]:
def calculate_fee(minutes_dialed):
    return 10 + 2 * minutes_dialed

for x in [1.0, 3.0, 5.0, 9.0, 10.0, 20.0]:
    print(calculate_fee(x))

接下来，我们把问题稍微变换一下，现在我们知道乘客每次乘坐出租车的公里数，也知道乘客每次下车的时候支付给出租车司机的总费用。但是并不知道乘车的起步价，以及每公里行驶费用是多少。我们希望让机器从这些数据当中学习出来计算总费用的规则。

更具体的，我们想要让机器学习程序通过数据学习出来下面的公式当中的参数w和参数b（这是一个非常简单的示例，所以`w`和`b`都是浮点数，随着对深度学习了解的深入，你将会知道`w`和`b`通常情况下会是矩阵和向量）。这样，当下次乘车的时候，我们知道了行驶里程`distance_travelled`的时候，我们就可以估算出来用户的总费用`total_fee`了。

```
total_fee = w * distance_travelled + b
```

接下来，我们看看用飞桨如何实现这个hello, world级别的机器学习程序。

# 导入飞桨

为了能够使用飞桨，我们需要先用python的`import`语句导入飞桨`paddle`，以及飞桨当中的核心计算引擎`paddle.fluid`。因为我们计划查看计算出来的参数，所以我们也会导入`paddle.fluid.param_attr`。

同时，为了能够更好的对数组进行计算和处理，我们也还需要导入`numpy`。

如果你是在本机运行这个notebook，而且还没有安装飞桨，可以去飞桨的官网查看如何安装：[飞桨官网](https://www.paddlepaddle.org.cn/)。并且请使用1.8或以上版本的飞桨。

In [0]:
import numpy as np
import paddle
import paddle.fluid as fluid
from paddle.fluid.param_attr import ParamAttr

"paddle version " + paddle.__version__

# 准备数据

在这个机器学习任务中，我们已经知道了乘客的行驶里程`distance_travelled`，和对应的，这些乘客的总费用`total_fee`。
通常情况下，在机器学习任务中，像`distance_travelled`这样的输入值，一般被称为`x`（或者特征`feature`），像`total_fee`这样的输出值，一般被称为`y`（或者标签`label`)。

我们用numpy当中的数组来表示这两组数据，然后用`np.hstack`把这两组数据拼成最终的数据。

In [0]:
x_data = np.array([[1.], [3.0], [5.0], [9.0], [10.0], [20.0]]).astype('float32')
y_data = np.array( [[12.], [16.0], [20.0], [28.0], [30.0], [50.0]]).astype('float32')
data = np.hstack((x_data, y_data))
print(data)

# 用飞桨定义神经网络(组网)

使用飞桨定义神经网络的过程，本质上，是我们用python，通过飞桨提供的API，来告诉飞桨我们的计算规则的过程。回顾一下，我们想要通过飞桨用机器学习方法，从数据当中学习出来如下公式当中的`w`和`b`。这样在未来，给定`x`时就可以估算出来`y`值（估算出来的`y`记为`y_predict`）

```
y_predict = w * x + b
```

如果用更加技术性的语言来说这件事，就是我们要使用一个只有一个神经元（`size=1`），并且没有激活函数(`act=None`)的全连接层(`fluid.layers.fc`)，来在输入为`x`的情况下计算`y_predict`。

这个公式里的变量`x, y, w, b, y_predict`，对应着飞桨里面的[Variable概念](https://www.paddlepaddle.org.cn/documentation/docs/zh/beginners_guide/basic_concept/variable.html)。

通过飞桨提供的API告诉飞桨我们想要学习的规则（机器学习模型）的方法，是依次的定义这些`Variable`，或者，给出这些`Variable`的计算方法。
`x, y`的定义对应着下面的代码片段的前两行代码，`y_predict`的计算，对应着下面的最后一行代码。

需要注意的是，全连接层(`fluid.layers.fc`)是一个含有需要学习的参数的操作(`OP`)，这些参数，也就是上面的公式当中的`w`和`b`，因此，`w`和`b`是在这个OP当中创建的，为了能告诉飞桨`w`和`b`的属性（例如，如何初始化，针对该参数的学习率，等等)，以及，在后续的代码中能够访问`w`和`b`，我们在调用这个`OP`的时候，指定了`w`和`b`的`ParamAttr`。如果想要了解关于 `OP`的更多的概念，可以参考[这里](https://www.paddlepaddle.org.cn/documentation/docs/zh/beginners_guide/basic_concept/operator.html#operator)。

### 稍微补充一下

在这里的示例中，我们根据经验，已经事先知道了`distance_travelled`和`total_fee`之间是线性的关系，而在更实际的问题当中，`x`和`y`的关系通常是非线性的，因此也就需要使用更多类型，也更复杂的神经网络。(比如，BMI指数跟你的身高就不是线性关系，一张图片里的某个像素值跟这个图片是猫还是狗也不是线性关系。）


In [19]:
x = fluid.data(name="x",shape=[None, 1],dtype='float32')
y = fluid.data(name="y",shape=[None, 1],dtype='float32')

w_attr = ParamAttr(name='w')
b_attr = ParamAttr(name='b')

y_predict = fluid.layers.fc(input=x, size=1, act=None, param_attr=w_attr, bias_attr=b_attr)

# 告诉飞桨怎么样学习

前面我们定义好了神经网络（尽管是一个最简单的神经网络），我们还需要告诉飞桨，怎么样去**学习**，从而能得到参数`w`和`b`。

这个过程简单的来陈述一下，你应该就会大致明白了（尽管背后的理论和知识还需要逐步的去学习）。在机器学习/深度学习当中，机器（计算机）在最开始的时候，得到参数`w`和`b`的方式是随便猜一下，用这种随便猜测得到的参数值，去进行计算（预测）的时候，得到的`y_predict`，跟实际的`y`值一定是有**差距**的。接下来，机器会根据这个差距来**调整`w`和`b`**，随着这样的逐步的调整，`w`和`b`会越来越正确，`y_predict`跟`y`之间的差距也会越来越小，从而最终能得到好用的`w`和`b`。这个过程就是机器**学习**的过程。

用更加技术的语言来说，衡量**差距**的函数（一个公式）就是损失函数，用来**调整**参数的方法就是优化算法。

在本示例当中，我们用最简单的均方误差(mean square error)作为损失函数，对应下面的前两行代码；和最常见的优化算法SGD（stocastic gradient descent)作为优化算法，对应下面的后两行代码。（传给`fluid.optimizer.SGD`的参数`learning_rate`，你可以理解为控制每次调整的步子大小的参数）。

In [0]:
loss = fluid.layers.square_error_cost(input=y_predict,label=y)
avg_loss = fluid.layers.mean(loss)

sgd_optimizer = fluid.optimizer.SGD(learning_rate=0.001)
sgd_optimizer.minimize(avg_loss)

# 准备好运行飞桨

飞桨是一个支持多种计算设备，和多种运行环境的深度学习框架，在真正开始运行优化算法之前，我们需要告诉飞桨在CPU设备上来运行，并且做必要的初始工作。关于这一部分的更多信息，请参考：[飞桨的基本概念](https://www.paddlepaddle.org.cn/documentation/docs/zh/beginners_guide/basic_concept/index_cn.html)。


In [0]:
cpu = fluid.CPUPlace()
exe = fluid.Executor(cpu)
exe.run(fluid.default_startup_program())

前面我们提到过，机器（计算机）在一开始的时候是随便猜`w`和`b`的，我们先看看机器猜的怎么样。你应该可以看到，这时候的`w`是一个随机值，`b`是0，这是飞桨的初始化策略。（如果你愿意，也可以采用其他的初始化的数字，今后你也会看到，选择不同的初始化策略也是对于做好机器学习任务来说很重要的一点）。

In [0]:
w_before_opt = fluid.global_scope().find_var("w").get_tensor()
b_before_opt = fluid.global_scope().find_var("b").get_tensor()

print("w before optimize")
print(np.array(w_before_opt))

print("b before optimize")
print( np.array(b_before_opt))

# 运行优化算法

接下来，我们让飞桨运行一下这个优化算法，这会是一个前面介绍过的逐步调整参数的过程，你应该可以看到loss值（衡量`y`和`y_predict`的差距的average loss)在不断的降低。

In [0]:
TOTAL_EPOCH = 5000
for i in range(TOTAL_EPOCH):
    # 每个epoch对数据进行随机打乱，会让优化算法更好的前进。
    np.random.shuffle(data)
    outs = exe.run(
        feed={'x':data[:,:1], 'y':data[:,1:]},
        fetch_list=[y_predict, avg_loss])
    if i%100 == 0:
        print("epoch {} average loss {}".format(i, outs[1][0]))
        
print("finished training， average loss {}".format(outs[1][0]))


# 机器学习出来的参数

经过了这样的对参数`w`和`b`的调整（**学习**)，我们再通过下面的程序，来看看现在的参数变成了多少。你应该会发现`w`变成了很接近2.0的一个值，`b`变成了接近10.0的一个值。虽然并不是正好的2和10，但却是从数据当中学习出来的还不错的模型的参数，可以在未来的时候，用从这批数据当中学习到的参数来预估了。（如果你愿意，也可以通过让机器多学习一段时间，从而得到更加接近2.0和10.0的参数值。)

In [0]:
w_after_opt = fluid.global_scope().find_var("w").get_tensor()
b_after_opt = fluid.global_scope().find_var("b").get_tensor()

print("w after optimize")
print(np.array(w_after_opt))

print("b after optimize")
print( np.array(b_after_opt))

# hello paddle

通过这个小教程，希望你已经初步了解了飞桨，能在接下来随着对飞桨的更多学习，来解决实际遇到的问题。

In [0]:
print("hello paddle")